In [ ]:
import pandas as pd
import altair as alt



First 5 records:    Rank                      Name Platform    Year         Genre Publisher  \
0     1                Wii Sports      Wii  2006.0        Sports  Nintendo   
1     2         Super Mario Bros.      NES  1985.0      Platform  Nintendo   
2     3            Mario Kart Wii      Wii  2008.0        Racing  Nintendo   
3     4         Wii Sports Resort      Wii  2009.0        Sports  Nintendo   
4     5  Pokemon Red/Pokemon Blue       GB  1996.0  Role-Playing  Nintendo   

   NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  
0     41.49     29.02      3.77         8.46         82.74  
1     29.08      3.58      6.81         0.77         40.24  
2     15.85     12.88      3.79         3.31         35.82  
3     15.75     11.01      3.28         2.96         33.00  
4     11.27      8.89     10.22         1.00         31.37  
First 5 records:    Unnamed: 0                        name  \
0           0                  Spider-Man   
1           1      Red Dead Redemption II

In [ ]:
raw_sales = pd.read_csv("Datasets/video games sales.csv")
raw_imdb = pd.read_csv("Datasets/imdb-videogames.csv")

df_sales = raw_sales[["Year", "Global_Sales"]].dropna()
df_imdb = raw_imdb[["year", "rating", "Action", "Adventure", "Comedy", "Crime", "Family", "Mystery", "Sci-Fi", "Thriller"]].dropna()

df_sales.head(5)
df_imdb.head(5)

,year,rating,Action,Adventure,Comedy,Crime,Family,Mystery,Sci-Fi,Thriller
0,2018.0,9.2,True,True,False,False,False,False,False,False
1,2018.0,9.7,True,True,False,True,False,False,False,False
2,2013.0,9.5,True,False,False,True,False,False,False,False
3,2018.0,9.6,True,True,False,False,False,False,False,False
4,2016.0,9.5,True,True,False,False,False,False,False,False
